# Function calling

https://platform.openai.com/docs/guides/function-calling

Function Calling은 OpenAI의 GPT 모델이 특정 작업을 수행할 수 있도록 함수 호출을 지원하는 기능이다.

이 기능을 활용하면 GPT 모델이 단순한 텍스트 생성뿐만 아니라 더 복잡한 작업도 자동으로 처리할 수 있다.

Function Calling을 통해 다음과 같은 작업을 수행할 수 있다:

1. **API 호출**: 외부 API를 호출하여 실시간 데이터를 가져오거나 특정 작업을 수행할 수 있다.  
   예) 날씨 정보 조회, 금융 데이터 조회

2. **데이터 처리**: 특정 함수를 호출하여 데이터 처리 작업을 수행할 수 있다.  
   예) 텍스트 데이터 분석, 통계 계산

3. **자동화된 작업**: 여러 단계의 작업을 자동화하여 수행할 수 있다.  
   예) 사용자가 요청한 내용을 기반으로 보고서 생성, 이메일 작성

**주요 특징**

- **함수 정의**: 사전에 정의된 함수나 API 엔드포인트를 GPT 모델에 통합할 수 있다.
- **자동 호출**: 사용자가 명시적으로 요청하지 않아도, GPT 모델이 컨텍스트를 이해하고 자동으로 적절한 함수를 호출할 수 있다.
- **입출력 처리**: 함수의 입력값을 자동으로 생성하고, 함수 호출 후 반환된 결과를 적절히 처리하여 사용자에게 전달할 수 있다.

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
OPENWEATHER_API_KEY = userdata.get('OPENWEATHER_API_KEY')


## Function 정의

In [ ]:
import requests
import json

def get_current_weather(city,units):
  url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units={units}'

  response = requests.get(url)
  data = response.json() # response content(json) 반환

  # llm에 전달할 날씨 정보
  if response.status_code == 200:
    weather_description = data['weather'][0]['description']
    temperature = data['main']['temp']
    humidity = data['main']['humidity']
    weather_info = {
        'location' : city,
        'weather':weather_description,
        'temperature':temperature,
        'humidity':humidity,
        'unit':'celsius' if units == 'metric' else 'fahrenheit'
    }
  else:
    # api 요청 실패한 경우
    weather_info = {
        'location' : city,
        'weather':'Not Found',
        'temperature':'Not Found',
        'humidity':'Not Found',
        'unit':'Not Found',
    }

  return json.dumps(weather_info)

get_current_weather('Seoul','metric')


'{"location": "Seoul", "weather": "broken clouds", "temperature": 23.79, "humidity": 86, "unit": "celsius"}'

## LLM 요청 흐름

In [ ]:
# LLM에게 각 function을 설명하는 메타 정보
from openai import OpenAI

tools = [{
    "type": "function",
    "function":{
        "name": "get_current_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨 정보를 얻고자 하는 도시 이름(영문)"
                },
                "units": {
                    "type": "string",
                    "enum": ["metric", "imperial"],
                    "description": "날씨에 관한 단위 설정, 섭씨를 사용하는 경우 metric, 화씨를 사용하는 경우 imperial 선택",
                }
            },
            "required": [
                "city"
            ],
            "additionalProperties": False
        }
    }
}]

messages = [
    {'role':'system', 'content':'당신은 친절한 챗봇으로 사용자의 요청에 상세히 답변하세요.'},
    {'role':'user', 'content':'오늘 서울 날씨를 알려줘'},
]
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
    tools = tools
)

response


ChatCompletion(id='chatcmpl-BmXGUOLKDJTvme50TR4np6azGmJa3', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rCp3JevwJCP0OokzPjfmKG6Z', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]))], created=1750907314, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=20, prompt_tokens=122, total_tokens=142, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
# LLM으로부터 받은 정보 확인하는 코드
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_rCp3JevwJCP0OokzPjfmKG6Z', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]

In [27]:
# 실행 가능한 함수 목록
available_functions = {
    "get_current_weather": get_current_weather,
}

# llm function call 응답 tool_calls 추가
messages.append(response_message)

# 각 함수 실행 및 응답 messages에 추가
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    print(function_response)

    # 응답을 메시지에 추가
    messages.append({
        'role': 'tool',
        'tool_call_id': tool_call.id,
        'name': function_name,
        'content': function_response
    })

# messages 확인용 출력
messages


{"location": "Seoul", "weather": "mist", "temperature": 23.76, "humidity": 94, "unit": "celsius"}


[{'role': 'system', 'content': '당신은 친절한 챗봇으로 사용자의 요청에 상세히 답변하세요.'},
 {'role': 'user', 'content': '오늘 서울 날씨를 알려줘'},
 {'role': 'assistant',
  'content': ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rCp3JevwJCP0OokzPjfmKG6Z', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')])},
 {'role': 'function',
  'tool_call_id': 'call_rCp3JevwJCP0OokzPjfmKG6Z',
  'name': 'get_current_weather',
  'content': '{"location": "Seoul", "weather": "mist", "temperature": 22.76, "humidity": 94, "unit": "celsius"}'},
 {'role': 'assistant',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_rCp3JevwJCP0OokzPjfmKG6Z', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]},
 {'role': 'tool',
  'tool_call_id': 'call_rCp3JevwJCP0OokzPjfmKG6Z',
  'name': 'get_current_weather

In [28]:
# 최종 llm 응답
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
)

response.choices[0].message.content


BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[2].content': expected one of a string or array of objects, but got an object instead.", 'type': 'invalid_request_error', 'param': 'messages[2].content', 'code': 'invalid_type'}}

In [29]:
# tool_call을 포함한 함수
def run_conversation(prompt):
  messages = [
      {'role':'system', 'content':'당신은 친절한 챗봇으로 사용자의 요청에 상세히 답변하세요.'},
      {'role':'user', 'content':prompt},
  ]
  client = OpenAI(api_key=OPENAI_API_KEY)

  # 첫번째 LLM 요청
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages = messages,
      tools = tools
  )
  # 첫번째 LLM 응답 확인
  response_message = response.choices[0].message
  tool_calls = response_message.tool_calls

  # tool_call이 필요한 경우와 아닌 경우 분기
  if tool_calls:
    messages.append(response_message)

    # 각 함수 실행 및 응답 messages에 추가
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)

        messages.append({
            'role': 'tool',
            'tool_call_id': tool_call.id,
            'name': function_name,
            'content': function_response
        })

      # 두 번째 LLM 요청
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
    )

  return response.choices[0].message.content
run_conversation('밥 먹었니?')


'저는 음식이 필요 없지만, 여러분의 식사 시간은 어떤지 궁금하네요! 오늘 어떤 음식을 드셨나요?'